In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss

np.random.seed(42)

In [ ]:
Q = np.array([[4.0, 0.0],
              [0.0, 1.0]])
b = np.array([0.0, 0.0])  # tomar b=0 para simplificar; el mínimo está en 0

def f(w):
    return 0.5 * w @ Q @ w + b @ w

def grad_f(w):
    return Q @ w + b

w0 = np.array([2.0, 2.0])
etas = [0.2, 0.5, 1.2]  # probar estable, crítico-ish, e inestable
steps = 30

fig, ax = plt.subplots(1, len(etas), figsize=(14, 4))
for j, eta in enumerate(etas):
    w = w0.copy()
    traj = [w.copy()]
    vals = [f(w)]
    diverged = False
    for k in range(steps):
        w = w - eta * grad_f(w)
        traj.append(w.copy())
        vals.append(f(w))
        if np.linalg.norm(w) > 1e6:  # diverge guard
            diverged = True
            break
    traj = np.array(traj)
    # Contornos
    xs = np.linspace(-2.5, 2.5, 200)
    ys = np.linspace(-2.5, 2.5, 200)
    X, Y = np.meshgrid(xs, ys)
    Z = 0.5*(4*X**2 + 1*Y**2)
    ax[j].contour(X, Y, Z, levels=10, linewidths=0.8, colors='gray')
    ax[j].plot(traj[:,0], traj[:,1], '-o', ms=3)
    ax[j].set_title(f"GD en cuadrática (η={eta})" + (" (diverge)" if diverged else ""))
    ax[j].set_xlabel("$w_1$")
    ax[j].set_ylabel("$w_2$")
plt.tight_layout()
plt.show()


In [ ]:
X, y = make_classification(n_samples=600, n_features=2, n_informative=2,
                           n_redundant=0, n_clusters_per_class=1, class_sep=1.5, random_state=0)
y = 2*y - 1  # convertir a {-1, +1}
X = StandardScaler().fit_transform(X)

def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def logistic_loss_grad(w, X, y, lam=1e-2):
    # f(w) = (1/n) sum log(1+exp(-y w^T x)) + lam/2 ||w||^2
    n = X.shape[0]
    z = y * (X @ w)
    loss = np.mean(np.log(1 + np.exp(-z))) + 0.5*lam*np.dot(w, w)
    # grad = -(1/n) sum [ y x * sig(-z) ] + lam w
    g = -(X.T @ (y * (1 - sigmoid(z))))/n + lam*w
    return loss, g

w = np.zeros(X.shape[1])
eta = 0.5
epochs = 200
history = []

for t in range(epochs):
    loss, g = logistic_loss_grad(w, X, y, lam=1e-2)
    w = w - eta*g
    history.append(loss)

print("Loss final:", history[-1])

# Curva de entrenamiento
plt.figure(figsize=(5,3))
plt.plot(history)
plt.xlabel("Iteración")
plt.ylabel("Función de pérdida")
plt.title("GD en regresión logística (L2=1e-2)")
plt.grid(True, alpha=0.3)
plt.show()

# Frontera de decisión
xx, yy = np.meshgrid(np.linspace(X[:,0].min()-1, X[:,0].max()+1, 200),
                     np.linspace(X[:,1].min()-1, X[:,1].max()+1, 200))
Z = (np.c_[xx.ravel(), yy.ravel()] @ w).reshape(xx.shape)
plt.figure(figsize=(4,4))
plt.contourf(xx, yy, (Z>0).astype(int), alpha=0.15, levels=1)
plt.contour(xx, yy, Z, levels=[0], linewidths=2)
plt.scatter(X[:,0], X[:,1], c=(y>0), s=15, cmap='bwr', edgecolor='k')
plt.title("Frontera GD (logística)")
plt.show()
